# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,61.00,83,76,1.92,AU,1611943324
1,Cape Town,-33.9258,18.4232,73.40,56,0,13.80,ZA,1611942913
2,Avarua,-21.2078,-159.7750,78.80,89,40,5.75,CK,1611943097
3,Uglovoye,44.8215,33.6045,44.01,62,99,7.00,UA,1611943325
4,Chui,-33.6971,-53.4616,79.63,59,52,10.56,UY,1611942967
...,...,...,...,...,...,...,...,...,...
572,Naze,28.3667,129.4833,48.69,73,65,14.07,JP,1611943155
573,Tonstad,58.6626,6.7169,32.00,74,0,3.44,NO,1611943428
574,Tessalit,20.1986,1.0114,72.77,17,0,9.46,ML,1611943125
575,Panzhihua,26.5851,101.7128,45.00,53,0,0.49,CN,1611943429


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key = g_key)
locations = weather_df[['Lat', 'Lng']]
humidity = weather_df['Humidity']

In [7]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity=100, point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_con_df = weather_df.loc[(weather_df["Wind Speed"] <= 15) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 75) & (weather_df["Max Temp"] <= 80)].dropna()
ideal_con_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Lima,-12.0432,-77.0282,78.80,57,0,8.05,PE,1611943059
107,Taoudenni,22.6783,-3.9836,78.57,16,0,6.44,ML,1611943343
217,Adrar,20.5022,-10.0711,77.34,16,0,7.70,MR,1611943123
300,Araouane,18.9048,-3.5265,77.45,16,0,7.96,ML,1611943156
350,Edd,13.9297,41.6925,78.93,43,0,13.87,ER,1611943390
442,Xai-Xai,-25.0519,33.6442,76.86,88,0,5.30,MZ,1611943405
473,Adeje,28.1227,-16.7260,78.01,68,0,9.22,ES,1611943366
547,Pisco,-13.7000,-76.2167,77.00,69,0,9.22,PE,1611943114


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = ideal_con_df.loc[:,['City', 'Country', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
5,Lima,PE,-12.0432,-77.0282,
107,Taoudenni,ML,22.6783,-3.9836,
217,Adrar,MR,20.5022,-10.0711,
300,Araouane,ML,18.9048,-3.5265,
350,Edd,ER,13.9297,41.6925,
442,Xai-Xai,MZ,-25.0519,33.6442,
473,Adeje,ES,28.1227,-16.7260,
547,Pisco,PE,-13.7000,-76.2167,


In [21]:
#base & params
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'type' : 'lodging',
          'radius' : 5000,
          'key' : g_key,
}

#for loop for nearby
for idx, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params['location'] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {idx}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
   
    #try/except
    try:
        hotel_df.loc[idx, 'Hotel Name'] = results[0]['name']
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        


    except (KeyError, IndexError):
        print('Missing value...skipping.')
        
    print('_______________')


print('_______Finsihed_______')

Retrieving Results for Index 5: Lima.
Closest hotel in Lima is Gran Hotel Bolivar Lima.
_______________
Retrieving Results for Index 107: Taoudenni.
Missing value...skipping.
_______________
Retrieving Results for Index 217: Adrar.
Missing value...skipping.
_______________
Retrieving Results for Index 300: Araouane.
Missing value...skipping.
_______________
Retrieving Results for Index 350: Edd.
Missing value...skipping.
_______________
Retrieving Results for Index 442: Xai-Xai.
Closest hotel in Xai-Xai is Complexo da Familia.
_______________
Retrieving Results for Index 473: Adeje.
Closest hotel in Adeje is Gran Tacande Wellness & Relax.
_______________
Retrieving Results for Index 547: Pisco.
Closest hotel in Pisco is Hotel San Isidro Oficial.
_______________
_______Finsihed_______


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
